In [1]:
!python --version

Python 3.6.9


In [3]:
# !pip install -q -U pip
# !pip install -q numpy
# !pip install -q pandas
!pip install -q ckiptagger
# !pip install -q tqdm
# !pip install -q tensorflow==1.14.0
# !pip install -q ipywidgets

In [4]:
import pandas as pd
import numpy as np

from ckiptagger import WS, POS
from tqdm.notebook import tqdm

In [5]:
df_train = pd.read_csv('news_clustering_train.tsv', sep='\t')
df_test = pd.read_csv('news_clustering_test.tsv', sep='\t')

In [6]:
df_train

,index,class,title
0,0,體育,亞洲杯奪冠賠率：日本、伊朗領銜 中國竟與泰國並列
1,1,體育,9輪4球本土射手僅次武磊 黃紫昌要搶最強U23頭銜
2,2,體育,如果今年勇士奪冠，下賽季詹姆斯何去何從？
3,3,體育,超級替補！科斯塔本賽季替補出場貢獻7次助攻
4,4,體育,騎士6天里發生了啥？從首輪搶七到次輪3-0猛龍
...,...,...,...
1795,1795,遊戲,LOL：麻辣香鍋韓服Rank不合成打野刀？電刀巨魔新套路連勝中
1796,1796,遊戲,穩住，我們能贏！因為我們擁有這種強大的力量
1797,1797,遊戲,騰訊是怎樣毀掉《鬥戰神》這款可能成神的作品的？
1798,1798,遊戲,LOL你不知道的黑科技打法！


In [9]:
train_titles = {row['index']: row['title'] for _, row in df_train.iterrows()}
train_classes = {row['index']: row['class'] for _, row in df_train.iterrows()}

test_titles = {row['index']: row['title'] for _, row in df_test.iterrows()}
test_classes = {row['index']: row['class'] for _, row in df_test.iterrows()}

In [10]:
all_news_class = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']

# 斷詞 + POS

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
!cp ./drive/MyDrive/NLP_data/data.7z .

In [14]:
!7z x data.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 3659454979 bytes (3490 MiB)

Extracting archive: data.7z
--
Path = data.7z
Type = 7z
Physical Size = 3659454979
Headers Size = 691
Method = LZMA2:26
Solid = +
Blocks = 2

  0%      0% 8 - D07data/embedding.7z                               1% 8 - D07data/embedding.7z                               2% 8 - D07data/embedding.7z                               3% 8 - D07data/embedding.7z                               4% 8 - D07data/embedding.7z                

In [15]:
ws = WS('./D07data/')
pos = POS('./D07data/')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:903: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


In [17]:
train_title_cuts = {}
for index, title in tqdm(train_titles.items()):
    # YOUR CODE HERE
    word_s = ws([title])
    word_p = pos(word_s)
    # END YOUR CODE
    train_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [19]:
test_title_cuts = {}
for index, title in tqdm(test_titles.items()):
    # YOUR CODE HERE
    word_s = ws([title])
    word_p = pos(word_s)
    # END YOUR CODE
    test_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [20]:
train_title_cuts[120]

[('國腳', 'Na'),
 ('張呈棟', 'Nb'),
 ('：', 'COLONCATEGORY'),
 ('從', 'D'),
 ('沒', 'D'),
 ('想', 'VE'),
 ('過', 'Di'),
 ('自己', 'Nh'),
 ('會', 'D'),
 ('出', 'VC'),
 ('一', 'Neu'),
 ('本', 'Nf'),
 ('書', 'Na')]

# Bag of Words (BOW)

In [23]:
word2index = {}
index2word = {}
# 產生字與index對應的關係
# YOUR CODE HERE
n = 0
for index in train_title_cuts:
  for word, flag in train_title_cuts[index]:
    if word in word2index:
      continue
    word2index[word] = n
    index2word[n] = word 
    n += 1

# END YOUR CODE

In [24]:
word2index['溫暖']

1520

In [25]:
index2word[1520]

'溫暖'

In [26]:
def get_bow_vector(pairs, word2index):
    # YOUR CODE HERE
    vector = np.zeros(len(word2index))
    for word, flag in pairs:
      if word in word2index:
        vector[word2index[word]] += 1

    # END YOUR CODE
    return vector

In [27]:
get_bow_vector(train_title_cuts[120], word2index)

array([0., 0., 0., ..., 0., 0., 0.])

# 排除較無意義的詞性

In [28]:
pos_analysis = {}
for _, pairs in train_title_cuts.items():
    for word, flag in pairs:
        if flag not in pos_analysis:
            pos_analysis[flag] = set()
        pos_analysis[flag].add(word)

for flag, words in pos_analysis.items():
    print(flag, ':', list(words)[:100])
    print('=======================')

Nb : ['扎堆', '趙雲', '卡納瓦羅', '力帆', '伯德', '建聯', '維韋杜詹霍', '本澤馬', '張本智和', '迪士尼', '帕克成', '拳皇', '游玄德', '弗格森', '劉國梁', '李白', '男乒', '雲聯惠', '埃姆雷', '王楚欽', '惠若琪', '凱爾特', '卓爾', '恆大', '朱嘯虎', '馬克', '綠軍', 'm15', '景泰石', '猛', '卡帥', '招行', '小強', '哈登', '大智慧', '世界杯', '賈躍亭', '亞亞·圖雷', '卡拉斯科', '餘額寶', '谷歌', '劉集', '夢琪', '萊萬', '秦緒文', '維亞利', '恆生', '埃爾克森', '瓜迪奧拉', '蓋倫', '安卓', '陶華碧', '堪比比薩', '多浪', '章澤天', '四阿哥', '勒布朗詹姆斯', '隆多', '德羅', '泰倫', '盼保', '林員', 'G3，76', 'S10', '比格利亞', '李嘉誠', '聯發科', '裴擒虎', '伊亞', '郎平', '奧尼爾', '楊智', '埃梅里', '里皮', '許', '德甲', '海因克斯 萊萬多夫斯基', '埃米爾岡', '溫格', '老詹', '樂福', '龔方雄', '張大仙', '騰訊', '馬龍', '韓信', '李盈瑩', '胤禛', '丘誠', '刷安徒恩', '加威廉', '權健', '安迪爾森', '宇通', '內馬爾', '英超', '蘇群', '黃希揚', '韓長賦', '洛瑞']
VC : ['誘', '生產', '撿', '測', '飲', '搭配', '關掉', '參觀', '開啓', '面試', '吸收', '踐行', '拿下', '零售', '通策', '計算', '破除', '考', '觀賞', '燒烤', '打理', '實施', '體驗', '爆存', '毀掉', '解決', '重演', '關閉', '衝刺', '攢', '投身', '增收', '燒', '指認', '包里', '產', '操作', '服務', '數', '扣', '唱', '掛', '弄', '看待', '反擊', '影響', '催收', '助推', '叫出', '動用', '等到', '佔領', '拍完', '玩上', '

|         Type        |     Description    |
|:-------------------:|:------------------:|
| A                   | 非謂形容詞         |
| Caa                 | 對等連接詞         |
| Cab                 | 連接詞，如：等等   |
| Cba                 | 連接詞，如：的話   |
| Cbb                 | 關聯連接詞         |
| D                   | 副詞               |
| Da                  | 數量副詞           |
| Dfa                 | 動詞前程度副詞     |
| Dfb                 | 動詞後程度副詞     |
| Di                  | 時態標記           |
| Dk                  | 句副詞             |
| DM                  | 定量式             |
| I                   | 感嘆詞             |
| Na                  | 普通名詞           |
| Nb                  | 專有名詞           |
| Nc                  | 地方詞             |
| Ncd                 | 位置詞             |
| Nd                  | 時間詞             |
| Nep                 | 指代定詞           |
| Neqa                | 數量定詞           |
| Neqb                | 後置數量定詞       |
| Nes                 | 特指定詞           |
| Neu                 | 數詞定詞           |
| Nf                  | 量詞               |
| Ng                  | 後置詞             |
| Nh                  | 代名詞             |
| Nv                  | 名物化動詞         |
| P                   | 介詞               |
| T                   | 語助詞             |
| VA                  | 動作不及物動詞     |
| VAC                 | 動作使動動詞       |
| VB                  | 動作類及物動詞     |
| VC                  | 動作及物動詞       |
| VCL                 | 動作接地方賓語動詞 |
| VD                  | 雙賓動詞           |
| VF                  | 動作謂賓動詞       |
| VE                  | 動作句賓動詞       |
| VG                  | 分類動詞           |
| VH                  | 狀態不及物動詞     |
| VHC                 | 狀態使動動詞       |
| VI                  | 狀態類及物動詞     |
| VJ                  | 狀態及物動詞       |
| VK                  | 狀態句賓動詞       |
| VL                  | 狀態謂賓動詞       |
| V_2                 | 有                 |
|                     |                    |
| DE                  | 的之得地           |
| SHI                 | 是                 |
| FW                  | 外文               |
|                     |                    |
| COLONCATEGORY       | 冒號               |
| COMMACATEGORY       | 逗號               |
| DASHCATEGORY        | 破折號             |
| DOTCATEGORY         | 點號               |
| ETCCATEGORY         | 刪節號             |
| EXCLAMATIONCATEGORY | 驚嘆號             |
| PARENTHESISCATEGORY | 括號               |
| PAUSECATEGORY       | 頓號               |
| PERIODCATEGORY      | 句號               |
| QUESTIONCATEGORY    | 問號               |
| SEMICOLONCATEGORY   | 分號               |
| SPCHANGECATEGORY    | 雙直線             |
| WHITESPACE          | 空白               |

In [29]:
def get_bow_vector_with_selection(pairs, word2index):
    excluded_flags = [
        # 根據以上列舉出來的文字以及詞性表，請列出想要排除的詞性
        # YOUR CODE HERE
        'Nh', 'Nep', 'Nes', 'DE', 'T', 'P', 'V_2', 'SHI',
        'Dfa', 'Dfb', 'Da', 'Di', 'Dk',
        'Caa', 'Cab', 'Cba', 'Cbb',
        'COLONCATEGORY', 'COMMACATEGORY', 'DASHCATEGORY', 'DOTCATEGORY', 'ETCCATEGORY', 'EXCLAMATIONCATEGORY',
        'PARENTHESISCATEGORY', 'PAUSECATEGORY', 'PERIODCATEGORY', 'QUESTIONCATEGORY', 'SEMICOLONCATEGORY',
        'SPCHANGECATEGORY', 'WHITESPACE'

        # END YOUR CODE
    ]
    vector = np.zeros(len(word2index))
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            vector[word2index[word]] += 1
    return vector

# Cosine Similarity

In [30]:
def cosine_similarity(bow1, bow2):
    # YOUR CODE HERE
    len_bow1 = np.sqrt(np.sum(np.square(bow1)))
    len_bow2 = np.sqrt(np.sum(np.square(bow2)))
    similarity = np.sum(bow1 * bow2) / (len_bow1 * len_bow2)
    # END YOUR CODE
    return similarity

In [31]:
bow1 = get_bow_vector(train_title_cuts[100], word2index)
bow2 = get_bow_vector(train_title_cuts[130], word2index)
cosine_similarity(bow1, bow2)

0.08703882797784893

In [32]:
train_title_cuts[100]

[('山東', 'Nc'),
 ('魯能', 'Nb'),
 ('有沒有', 'D'),
 ('可能', 'D'),
 ('拿到', 'VC'),
 ('今年', 'Nd'),
 ('的', 'DE'),
 ('中', 'A'),
 ('超', 'A'),
 ('冠軍', 'Na'),
 ('？', 'QUESTIONCATEGORY')]

In [33]:
train_title_cuts[130]

[('NBA', 'Nb'),
 ('和', 'Caa'),
 ('CBA', 'FW'),
 ('差距', 'Na'),
 ('在', 'P'),
 ('哪裡', 'Ncd'),
 ('？', 'QUESTIONCATEGORY'),
 ('6', 'Neu'),
 ('張', 'Nf'),
 ('圖', 'VF'),
 ('一目瞭然', 'VH'),
 ('！', 'EXCLAMATIONCATEGORY')]

# Group mean vector

In [34]:
group_vectors = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(train_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    news_class = train_classes[index]
    group_vectors[news_class].append(vector)

group_mean_vector = {}
for news_class, vectors in group_vectors.items():
    group_mean_vector[news_class] = np.mean(vectors, axis=0)
group_mean_vector

{'旅遊': array([0., 0., 0., ..., 0., 0., 0.]),
 '科技': array([0., 0., 0., ..., 0., 0., 0.]),
 '財經': array([0., 0., 0., ..., 0., 0., 0.]),
 '農業': array([0., 0., 0., ..., 0., 0., 0.]),
 '遊戲': array([0.        , 0.        , 0.        , ..., 0.00333333, 0.00333333,
        0.00333333]),
 '體育': array([0.04      , 0.01333333, 0.00333333, ..., 0.        , 0.        ,
        0.        ])}

# Group mean vector: 測試

In [35]:
classification = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(test_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    if np.sum(np.square(vector)) == 0:
        continue

    max_val = -2.0
    max_class = None
    for news_class, ref_vector in group_mean_vector.items():
        val = cosine_similarity(ref_vector, vector)
        if val > max_val:
            max_class = news_class
            max_val = val

    classification[max_class].append(index)

In [36]:
from collections import Counter

for group, ids in classification.items():
    counter = Counter([test_classes[id] for id in ids])
    print('predict', group, ': ', counter)

predict 體育 :  Counter({'體育': 73, '遊戲': 8, '財經': 6, '旅遊': 6, '農業': 6, '科技': 3})
predict 財經 :  Counter({'財經': 67, '科技': 18, '體育': 6, '旅遊': 6, '遊戲': 6, '農業': 5})
predict 科技 :  Counter({'科技': 62, '財經': 13, '體育': 10, '農業': 6, '遊戲': 6, '旅遊': 2})
predict 旅遊 :  Counter({'旅遊': 70, '農業': 11, '財經': 6, '科技': 3, '遊戲': 2})
predict 農業 :  Counter({'農業': 70, '旅遊': 9, '科技': 6, '體育': 5, '財經': 4, '遊戲': 3})
predict 遊戲 :  Counter({'遊戲': 75, '科技': 7, '旅遊': 5, '體育': 4, '財經': 4, '農業': 1})
